# Let's explore how to extract language features with KEVIN'S CODE

test_ego4d_mq_l.py --r $egovlp_checkpoint --tokens --save_feats xxx --gpu x

In [1]:
import argparse
import torch
import transformers
import os
import json
import inspect
from pprint import pprint

import model.model as module_arch
from utils import state_dict_data_parallel_fix

/home/soldanm/anaconda3/envs/ego/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'utils.frozen_ego_minimum'

In [ ]:
def initialize(config, name, module,  *args, **kwargs):
    """
    finds a function handle with the name given as 'type' in config, and returns the
    instance initialized with corresponding keyword args given as 'args'.
    """
    module_name = config[name]['type']
    module_args = dict(config[name]['args'])
    assert all([k not in module_args for k in kwargs]), 'Overwriting kwargs given in config file is not allowed'
    module_args.update(kwargs)

    return getattr(module, module_name)(*args, **module_args)

In [12]:
# build model architecture
config = json.load(open('/home/soldanm/Downloads/EgoVLP_minimal/frozen-ego-minimum/configs/ego4d-nlq-l.json','r'))
model = initialize(config, 'arch', module_arch) 
checkpoint = './pretrained/frozen_pt_egoclip.pth'
if os.path.exists(checkpoint):
    checkpoint = torch.load(checkpoint)
    state_dict = checkpoint['state_dict']
    new_state_dict = state_dict_data_parallel_fix(state_dict, model.state_dict())
    model.load_state_dict(new_state_dict, strict=True)
else:
    raise ValueError('Checkpoint not found')
    
# prepare model for testing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

# Build tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


######USING ATTENTION STYLE:  frozen-in-time


In [31]:
text = ['I have 4 apples']
with torch.no_grad():
    assert len(text)==1
    text = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    text = {key: val.cuda() for key, val in text.items()}
    text_embed = model.compute_text_tokens(text)[0]
    num_words = text['attention_mask'][0].sum()
    text_embed = text_embed[1 : num_words-1]
    print(text_embed.detach().cpu())

tensor([[-0.0942, -0.0557,  0.3260,  ...,  0.5542,  0.5677,  0.0899],
        [-0.1507, -0.0817,  0.3609,  ...,  0.6262,  0.5927,  0.0729],
        [-0.1331, -0.0289,  0.4076,  ...,  0.6634,  0.6799,  0.1265],
        [-0.0615,  0.0102,  0.4534,  ...,  0.6683,  0.5676,  0.0849]])


In [2]:
import torch
# cc3mwebvid.tar
# egoclip_egonce.pth
# egovlp.pth
# ht100m
checkpoint = './pretrained/ht100m.pth'
checkpoint = torch.load(checkpoint)
checkpoint = {'state_dict': checkpoint['state_dict']}
torch.save(checkpoint, './pretrained/ht100m.pth')

ModuleNotFoundError: No module named 'parse_config_dist_multi'

In [1]:
import torch
checkpoint = './pretrained/ht100m.pth'
print(torch.load(checkpoint).keys())

/home/soldanm/anaconda3/envs/ego/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'parse_config_dist_multi'